In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir())

# Any results you write to the current directory are saved as output.

['utils.py', 'dataset', 'generate_dataset.py', 'model', 'generate_missing_data.py', 'GAN.ipynb', '.ipynb_checkpoints', 'fill_dataset.ipynb', 'MLP_Classifier.ipynb', 'generate_csv_files.ipynb', 'LICENSE', 'generate_ones.py', '.git', 'README.md', '.gitignore']


In [2]:
from sklearn.utils import shuffle 

df = pd.read_csv("dataset/train.csv", index_col=0)
df = shuffle(df)
df.sample(2)

,target,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
289,5,0.443137,0.494118,0.443137,0.505882,0.411765,0.501961,0.533333,0.521569,0.505882,...,0.458824,0.458824,0.505882,0.380392,0.470588,0.470588,0.423529,0.501961,0.450980,0.501961
7,0,0.392157,0.356863,0.329412,0.360784,0.349020,0.368627,0.325490,0.325490,0.356863,...,0.345098,0.333333,0.372549,0.349020,0.341176,0.360784,0.388235,0.368627,0.301961,0.360784


In [3]:
y = df.iloc[:,:1]
x = df.iloc[:,1:]

# Input data

In [4]:
from math import ceil

n = x.shape[0]
n_train = ceil(0.8 * n)
n_test = ceil(0.2 * n)

x_train = x[:n_train]
y_train = y[:n_train]

x_test = x[n_train:]
y_test = y[n_train:]

y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)

# Building the model and evaluating

In [5]:
from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(solver='adam', 
                          alpha=1e-5, 
                          hidden_layer_sizes=(1024, 128), 
                          random_state=1, 
                          max_iter=150)

print(mlp_model)

mlp_model.fit(x_train, y_train)                         
pred_mlp =  mlp_model.predict(x_test)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1024, 128), learning_rate='constant',
       learning_rate_init=0.001, max_iter=150, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)


/home/iepsen/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [6]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics

print("\n\nTesting using MLP")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, pred_mlp))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, pred_mlp))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, pred_mlp))) 



Testing using MLP
Mean Absolute Error: 0.39
Mean Squared Error: 2.31
Root Mean Squared Error: 1.5198684153570663


In [7]:
print("\n\nMLP")
print("--- confusion_matrix ---")
print(confusion_matrix(y_test, pred_mlp))  
print("\n--- classification report ---")
print(classification_report(y_test, pred_mlp))  
print("\nmodel accuracy: ", accuracy_score(y_test, pred_mlp))



MLP
--- confusion_matrix ---
[[12  0  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  0  0  0  0  1  0]
 [ 0  0 10  0  0  0  0  0  0  0]
 [ 0  0  1  8  0  0  0  0  0  1]
 [ 0  0  0  0 13  0  0  0  0  0]
 [ 0  0  0  0  0  9  0  0  0  0]
 [ 1  0  0  0  0  1  8  0  0  0]
 [ 0  0  0  0  0  0  0  9  0  0]
 [ 0  0  1  0  0  0  0  0 10  0]
 [ 0  0  0  2  0  0  0  0  0  6]]

--- classification report ---
             precision    recall  f1-score   support

          0       0.92      1.00      0.96        12
          1       1.00      0.88      0.93         8
          2       0.83      1.00      0.91        10
          3       0.80      0.80      0.80        10
          4       1.00      1.00      1.00        13
          5       0.90      1.00      0.95         9
          6       1.00      0.80      0.89        10
          7       1.00      1.00      1.00         9
          8       0.91      0.91      0.91        11
          9       0.86      0.75      0.80         8

avg / total       0.92

# Recognizing numbers (or not)

In [8]:
df_test = pd.read_csv("dataset/test.csv", index_col=0)
df_test = shuffle(df_test)
df_test.sample(2)

,target,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
62,6,0.454902,0.450980,0.466667,0.443137,0.411765,0.498039,0.494118,0.505882,0.470588,...,0.423529,0.411765,0.482353,0.388235,0.400000,0.443137,0.427451,0.462745,0.411765,0.474510
96,9,0.466667,0.443137,0.435294,0.388235,0.403922,0.494118,0.470588,0.458824,0.474510,...,0.396078,0.364706,0.466667,0.376471,0.380392,0.458824,0.388235,0.482353,0.396078,0.490196


In [9]:
test = df_test.iloc[:, 1:].values
test_targets = df_test.iloc[:, 0].values
test 

array([[0.44313725, 0.41960784, 0.43529412, ..., 0.42745098, 0.36078431,
        0.45098039],
       [0.29411765, 0.32941176, 0.29411765, ..., 0.29411765, 0.27843137,
        0.24313725],
       [0.39215686, 0.40784314, 0.38039216, ..., 0.38823529, 0.3372549 ,
        0.39215686],
       ...,
       [0.47058824, 0.44313725, 0.43137255, ..., 0.4627451 , 0.41176471,
        0.42352941],
       [0.30980392, 0.2745098 , 0.28627451, ..., 0.28627451, 0.3254902 ,
        0.2745098 ],
       [0.48235294, 0.48235294, 0.45882353, ..., 0.50196078, 0.41960784,
        0.45098039]])

In [10]:
preds =  mlp_model.predict(test)

In [11]:
print("\nmodel accuracy: ", accuracy_score(test_targets, preds))


model accuracy:  0.8585858585858586


# Testing with real numbers of dataset

In [12]:
df_real = pd.read_csv("dataset/mnist.csv")
df_real = shuffle(df)
df_real.sample(2)

,target,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
283,5,0.443137,0.423529,0.435294,0.454902,0.380392,0.431373,0.454902,0.431373,0.423529,...,0.431373,0.407843,0.439216,0.407843,0.431373,0.396078,0.419608,0.431373,0.388235,0.470588
385,7,0.231373,0.247059,0.262745,0.294118,0.184314,0.219608,0.235294,0.396078,0.278431,...,0.215686,0.203922,0.356863,0.250980,0.254902,0.262745,0.333333,0.270588,0.254902,0.196078


In [13]:
data_real = df_real.iloc[:,1:].values
targets_real = df_real.iloc[:,0].values
preds =  mlp_model.predict(data_real)

In [14]:
print("\nmodel accuracy: ", accuracy_score(targets_real, preds))


model accuracy:  0.984
